In [ ]:
!pip install -U -q transformers datasets accelerate peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 19.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import random
import os
import re
import shutil
import torch
from tqdm import tqdm

# Hugging Face Transformers ecosystem
from transformers import pipeline
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import TrainingArguments, Trainer
from datasets import Dataset

# Parameter-Efficient Fine-Tuning (PEFT)
from peft import get_peft_model, LoraConfig, TaskType
from peft import PeftModel

# Semantic similarity and embeddings
from sentence_transformers import SentenceTransformer, util

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# SECTION: Load Interview Questions CSV
# Load the dataset (update the filename/path as needed)
df = pd.read_csv('/content/Software Questions.csv',encoding='latin1')

# Show unique difficulty levels
print("Unique Difficulty Levels:", df['Difficulty'].unique())

# Preview dataset
df.head()

Unique Difficulty Levels: ['Medium' 'Hard' 'Easy']


,Question Number,Question,Answer,Category,Difficulty
0,1,What is the difference between compilation and...,Compilation translates source code into machin...,General Programming,Medium
1,2,Explain the concept of polymorphism.,Polymorphism allows objects of different class...,General Programming,Medium
2,3,Define encapsulation and give an example.,Encapsulation bundles data and methods in a cl...,General Programming,Hard
3,4,"What is an abstract class, and how is it diffe...",An abstract class can't be instantiated and ca...,General Programming,Medium
4,5,Describe the principles of Object-Oriented Pro...,"OOP principles include encapsulation, inherita...",General Programming,Medium


In [ ]:
# Load the dataset
df = pd.read_csv('/content/Software Questions.csv', encoding='latin1')

# Load a larger model for better generation quality
feedback_model = pipeline(
    "text2text-generation",
    model="google/flan-t5-large",  # More capable than t5-small
    do_sample=True,
    temperature=0.7,
    top_p=0.9
)

# Improved Prompt with clear feedback and edge case handling
few_shot_prompt = """
You are an AI interview coach. Your task is to evaluate a candidate's technical interview answer.

Provide the feedback in the following sections:
1. ✅ Strengths: List the strong points of the candidate's answer.
2. ❌ Weaknesses or Misconceptions: Identify areas where the answer is lacking or incorrect.
3. 📈 Suggestions for Improvement: Provide specific suggestions on how the candidate can improve their answer.
4. 🧠 Score: Give a score out of 10 based on the overall quality of the answer.

Make sure to fill each section with detailed and constructive feedback based on the candidate's answer.

If the candidate says "I don't know" or provides a vague or incomplete answer, kindly provide the correct answer or suggest areas where they can improve. Encourage learning and provide helpful feedback.

Example 1:
Question: What is the difference between compilation and interpretation?
Ideal Answer: Compilation translates source code into machine code creating an executable file. Interpretation translates and executes code line by line without an executable.
Candidate Answer: Compilation turns source code to machine code; interpretation runs code line by line.
Feedback:
✅ Strengths: The candidate correctly identified the key difference between compile-time and interpretation.
❌ Weaknesses or Misconceptions: The candidate missed that compilation creates an executable file.
📈 Suggestions for Improvement: The candidate could mention real-world examples like C (compiled) vs Python (interpreted).
🧠 Score: 7/10

Example 2:
Question: What is polymorphism?
Ideal Answer: Polymorphism allows methods to have different behaviors based on the object calling them.
Candidate Answer: It allows a function to behave differently.
Feedback:
✅ Strengths: The candidate has a basic understanding of polymorphism.
❌ Weaknesses or Misconceptions: The candidate's answer is vague and does not mention key aspects such as objects, overriding, or overloading.
📈 Suggestions for Improvement: The candidate could provide examples from object-oriented programming or use analogies to clarify the concept.
🧠 Score: 6/10

Now evaluate this:
"""

# Function to generate feedback using the new, clear prompt with edge case handling
def generate_feedback(question, candidate_answer, ideal_answer):
    # Edge case handling: Check if the answer is empty or something like "I don't know"
    if candidate_answer.strip().lower() in ['i don\'t know', 'na', '', 'no idea', 'dont know']:
        feedback = f"""
✅ Strengths: The candidate was honest in stating that they don't know the answer. This is an important trait in interviews.
❌ Weaknesses or Misconceptions: The candidate did not attempt to answer the question.
📈 Suggestions for Improvement: It would be helpful to study the key concepts related to this topic. Here's the correct answer:
{ideal_answer}
🧠 Score: 3/10 (No answer provided, but this can be improved with more preparation.)
"""
    # If the answer is valid but incomplete, provide suggestions to expand it
    elif len(candidate_answer.split()) < 10:
        feedback = f"""
✅ Strengths: The candidate has made an attempt to answer the question.
❌ Weaknesses or Misconceptions: The answer lacks detail and does not fully address the question.
📈 Suggestions for Improvement: Please elaborate more on the key concepts and provide additional examples or explanations.
🧠 Score: 5/10 (The answer is too brief, but it could be improved with more information.)
"""
    else:
        # Generate feedback using the model
        prompt = f"""{few_shot_prompt}
Question: {question}
Ideal Answer: {ideal_answer}
Candidate Answer: {candidate_answer}
Feedback:"""
        result = feedback_model(prompt, max_length=512, truncation=True)
        feedback = result[0]['generated_text'].strip()

    return feedback
# Live Q&A loop
while True:
    random_row = df.sample(n=1).iloc[0]
    question = random_row['Question']
    ideal_answer = random_row['Answer']

    print("\n🧠 Random Interview Question:")
    print(question)

    candidate_answer = input("\n👤 Your Answer (type 'stop' to quit): ")
    if candidate_answer.strip().lower() == 'stop':
        print("🛑 Session ended.")
        break

    print("\n✅ AI Feedback:")
    feedback = generate_feedback(question, candidate_answer, ideal_answer)
    print(feedback)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu



🧠 Random Interview Question:
Design and implement a large-scale distributed cache system.

👤 Your Answer (type 'stop' to quit): quit

✅ AI Feedback:

✅ Strengths: The candidate has made an attempt to answer the question.
❌ Weaknesses or Misconceptions: The answer lacks detail and does not fully address the question.
📈 Suggestions for Improvement: Please elaborate more on the key concepts and provide additional examples or explanations.
🧠 Score: 5/10 (The answer is too brief, but it could be improved with more information.)


🧠 Random Interview Question:
Design a secure mobile payment system for developing countries.

👤 Your Answer (type 'stop' to quit): quit

✅ AI Feedback:

✅ Strengths: The candidate has made an attempt to answer the question.
❌ Weaknesses or Misconceptions: The answer lacks detail and does not fully address the question.
📈 Suggestions for Improvement: Please elaborate more on the key concepts and provide additional examples or explanations.
🧠 Score: 5/10 (The answer

In [ ]:
import pandas as pd
import random
import os
from tqdm import tqdm
from transformers import pipeline

# Load your dataset
df = pd.read_csv("/content/Software Questions.csv",encoding='latin1')

# Load Flan-T5 large model
generator = pipeline("text2text-generation", model="google/flan-t5-large", max_length=200)

# Create diverse "I don't know" variations
dont_know_variants = [
    "I'm not sure about this one.",
    "I don't know the answer to this.",
    "Sorry, I can't recall this.",
    "I'm unsure, but I'd like to learn more about it.",
    "I don't know."
]

# Prepare your data
answer_data = []

def build_prompt(question):
    return f"You are a job candidate answering this interview question naturally and honestly:\nQuestion: {question}\nAnswer:"

for _, row in tqdm(df.iterrows(), total=len(df)):
    qid = row['Question Number']
    question = row['Question']
    ideal = row['Answer']
    difficulty = row['Difficulty']

    # Add one "I don't know" variation
    answer_data.append({
        "question_id": qid,
        "question": question,
        "candidate_answer": random.choice(dont_know_variants),
        "ideal_answer": ideal,
        "difficulty": difficulty,
        "answer_type": "unknown"
    })

    # Generate 4 realistic answers
    prompt = build_prompt(question)
    responses = generator(prompt, num_return_sequences=4, do_sample=True, top_p=0.9, temperature=0.7)

    for response in responses:
        answer_data.append({
            "question_id": qid,
            "question": question,
            "candidate_answer": response['generated_text'].strip(),
            "ideal_answer": ideal,
            "difficulty": difficulty,
            "answer_type": "humanlike"
        })

# Convert to DataFrame
gen_df = pd.DataFrame(answer_data)

# Save to CSV
gen_df.to_csv("/content/Software Questions.csv", index=False)

Device set to use cpu
100%|██████████| 200/200 [53:35<00:00, 16.08s/it]
